In [4]:
import functions as fc
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow.contrib.learn as learn
import tensorflow as tf
import numpy as np
import time

print("program start")
tf.logging.set_verbosity(tf.logging.ERROR)

path = "./data/"
print("reading in file")
filename_train = os.path.join(path,"Little_training_dataset.csv")
filename_test = os.path.join(path,"testing_dataset.csv")
filename_compare = os.path.join(path,"compare.csv")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])
#df_test = pd.read_csv(filename_test,na_values=['NA','?'])

print("preprocessing files")
fc.encode_numeric_zscore(df_train,'center_to_line')
fc.encode_numeric_zscore(df_train,'line_in_sphere')

x, y = fc.to_xy(df_train , 'line_in_cube')

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, random_state=42)

model_dir = fc.get_model_dir('LineInCube',True)

feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[1])]

regressor = learn.DNNRegressor(
    model_dir= model_dir,
    config=tf.contrib.learn.RunConfig(save_checkpoints_steps=500),
    feature_columns=feature_columns,
    hidden_units=[50, 25, 10])
    #hidden_units=[2, 3, 1])


validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    x_test,
    y_test,
    every_n_steps=500,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=50)


start_time = time.time()
print("start training")
regressor.fit(x_train, y_train,monitors=[validation_monitor],steps=10000)
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(fc.hms_string(elapsed_time)))
print("Best step: {}, Last successful step: {}".format(
validation_monitor.best_step,validation_monitor._last_successful_step))
# Predict
pred = list(regressor.predict(x_test, as_iterable=True))
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

program start
reading in file
preprocessing files
start training


Elapsed time: 0:00:11.02
Best step: 2000, Last successful step: 2500


Final score (RMSE): 0.14236295223236084


In [10]:
#from tensorflow.python.training import saver
#saver.
# Predict
tf.logging.set_verbosity(tf.logging.INFO)
pred = list(regressor.predict(x_test, as_iterable=True))
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

saver = learn.DNNRegressor(
    model_dir= model_dir,
    config=tf.contrib.learn.RunConfig(save_checkpoints_steps=500),
    feature_columns=feature_columns,
    hidden_units=[50, 25, 10])

pred = list(saver.predict(x_test, as_iterable=True))
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


INFO:tensorflow:Restoring parameters from ./dnn/LineInCube/model.ckpt-2500


Final score (RMSE): 0.14182955026626587
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x115a46748>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_session_config': None, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './dnn/LineInCube'}


Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


INFO:tensorflow:Restoring parameters from ./dnn/LineInCube/model.ckpt-2500


Final score (RMSE): 0.14182955026626587


In [15]:
import csv
fileHeader = ['mean','std']
center_to_line = [df_train['center_to_line'].mean(), df_train['center_to_line'].std()]
line_in_sphere = [df_train['line_in_sphere'].mean(), df_train['line_in_sphere'].std()]
csvFile = open("./param/param.csv","w")
writer = csv.writer(csvFile)
writer.writerow(fileHeader)
writer.writerow(center_to_line)
writer.writerow(line_in_sphere)
csvFile.close()

In [33]:
mean = [df_train['center_to_line'].mean(), df_train['line_in_sphere'].mean()]
std = [df_train['center_to_line'].std(), df_train['line_in_sphere'].std()]
content = {'mean':mean,'std':std}
df_param = pd.DataFrame(data = content)
df_param.to_csv("./param/param.csv",index=False)
print(df)
df['mean'][0]
type(df['mean'][0] + df['mean'][1])

           mean  std
0 -3.552714e-17  1.0
1 -1.232348e-16  1.0


numpy.float64